### Section 8.4.2 - Plain Enums

Enumerators, i.e., members of <b>plain enum</b> have their scope "leaked" to the same scope as of their enum. They are also implicitly converted to int, i.e., their value can be used in int's, even though the converse is not possible. E.g.:

In [1]:
#include<iostream>

enum Leaking{ 
    
    first_leak, second_leak
};

int get_value = second_leak; // example of scope leak

std::cout << get_value << "\n";

int receiver = Leaking::first_leak; // Example of implicit conversion from enum to int

std::cout << receiver << "\n";

// Leaking leak = 4 // This case is not possible.

1
0


@0x7f523de24b60

Because of this leakage problem, called <b>Namespace Pollution</b>, avoid common names for enumerators of <b>plain enum</b>. Try to use names that provide some info about the fact that the enumerator is indeed a member of a specific enum type. Disambiguation is the core idea behind this.

As for enum class, it is also possible to specify the <i>underlying integral type</i> of a <b>plain enum</b>; E.g.:

In [2]:
// Example of underlyint integral type specification
enum underlying_type : char{
    
    first, second, third
};

enum forward_declared : int; // Example of forward declaration of plain enum

This also make it possible for we to <b>forward declare</b> an enum, just like in the last line above. Without specifying an underlyint type, forward declarations are not allowed.

We said above that implicit conversions from int to plain enum is not possible, but this is not the case for <b>explicit</b> conversions. But, be careful, this is only <b>DEFINED</b> when the value of the int that will be used in the conversion to a plain enum is <b>within the range</b> of the enumeration. This range is defined as the smallest bit-field capable of holding the enumerators, i.e., members, values( almost a literal quote from Stroustrup ). This bit-field is calculated from the smallest k s.t. $2^k$( or $-2^k$ ) covers the range of values of the enumerators. If a value is <b>OUTSIDE</b> this range, then we have the famous <b>Undefined Behavior</b>.

A last note about conversions: since plain enum has an underlying integral type, there is indeed an <b>implicit conversion</b> from the former to the latter. This also means that

In [3]:
enum example{
    
    just_a_value
};

sizeof( example )

4

<b>enum example</b> has the <b>sizeof</b> its underlying, else explicitly specified, type. If the type is not explicityly specified, then the type will be of some integral type that can hold the range of values provided, where this type is no larger than <b>sizeof( int )</b>.

# Chapter 9 - Statements

## Section 9.1 - Introduction

Statements do not have values. Instead, they are used to specify the order of execution of the program.

## Section 9.2 - Statement Summary

There is a list of what are considered statements on page 226.

<b>Declarations</b> are statements whenever no initialization or procedure-calls are done. If one of the latter two happens, then we have an expression.

Sequences of statements made within curly braces are considered <b>block of code</b> or <b>compound statements</b>.

## Section 9.3 - Declarations As Statements

As said above, declarations are statements, and the initialization of a declared entity happens when the thread of control reaches the declared entity.

Consider defining user-defined types only when a suitable value is available, so as to initialize at construction. This takes less instructions to be performed, leading to better performance. Only consider initialization <i>After</i> declaration when it is not possible to do as previously stated, i.e., when the value for the entity depends on another statement. E.g.: when using

In [1]:
#include<iostream>

int value{};

std::cin >> value;

std::cout << value << "\n";

45
45


where <b>int value</b> value depends on the input of <b>std::cin</b>.

## Section 9.4 - Selection Statements

Related to <b>if( ), if( ) else, switch( )</b> and <b>else if( )</b> statements.

Conditions are either expressions or declarations.

### Section 9.4.1 - <span style="color: blue;">if</span> statements

Evaluates a condition and executes an or many instructions, statements, if the condition come out to be true. If the latter is not true, an <b>else</b> statement, if provided, will execute its own set of instructions.

Arguments used in conditions can be <b>implicitly converted</b> to booleans, e.g.: arithmetic expressions and pointer expressions. Yeah, I know, dangerous and shitty, but what can we do about it? Simple: avoid passing arguments that can  be implicitly converted!

<b>Note:</b> consider using expressions with types for which <b>explicit</b> constructros were provided. This avoids implicit conversions when dealing with user-defined types, and will possibly save you from some NASTY trouble.

Consider the following snippets, they explain how implicit conversions works for certain cases:

In [2]:
int x{ };

if( x ){ // This statemente is equivalent to
}

if( x != 0 ){ // this statement
}

int* pointer{ };

if( pointer ){ // This statement is equivalent to
}

if( pointer != nullptr ){ // This statement
}

enum implicit_conversion{
    
    condition_one, condition_two
};

implicit_conversion test;

if( test ){ // This statement is valid, because plain enums are implicitly convertible to int
}

enum class no_conversion{
    
    no_condition
};

no_conversion test_2;

/*if( test_2 ){ // This statement is not valid, because enum class is not implicitly convertible to int
}
*/

Binary logical operators like && and || only evaluates the second argument if necessary. I.e., they are <b>lazy</b>. An example would be an expression A && B, where A is false. It is not necessary for B to be evaluated at all, since the expression will be false whichever truth value B has.

<b>if statements'</b> block of code provides boundaries to entities declared in it. I.e., anything declared in an if-block cannot be used outside this block, not even in the else block associated to the if part. E.g.:

In [ ]:
if( true ){
    
    int value{ }; // value is bound to if scope. It cannot be accesses outside these brackets
    
    ++value;
}

else{
    
    // do something else
    
    // value--; // This is not possible, since value was not declared in here
}

But, for declarations to be possible in if-statements, they <b>must</b> be enclosed in blocks! Else, the declaration would "leak" out of the scope.

A possible question is: what is the purpose of opening an if-statement only to declare a name?

In [ ]:
if( int condition )
    
     int newval = condition + 1; // This is not possible

// The correct way of doing this is
if( int newcondition ){
    
    int declaration = newcondition;
}

### Section 9.4.2 - <span style="color: blue;">switch</span> Statements

Expressions is <b>case</b> labels must be of integral or enumeration type, and it is not possible to repeat values in the former. E.g.:

In [ ]:
int value{ };

switch( value ){
        
    case 2:
        
        break;
        
    case 2: // not allowed. Repeated value.
        
        break;
}

<b>switch</b> <b>if</b> statements are somewhat interchangeable. This means that either one can be written as the other. But, keep in mind: in some cases, it is preferred to use one insted of the other. For example, <b>switch</b> statements are more explicit when testing for constants, i.e., the semantic behind switch is more appropriate for those cases. Thus, consider the semantic when choosing between both <b>switch</b> and <b>if</b>. Also, in terms of performance, according to https://stackoverflow.com/questions/97987/advantage-of-switch-over-if-else-statement, <b>switch</b> <i>might</i> be faster than <b>if</b> statements.

Cases in <b>switch</b> statements must be explicitly terminated, unless we want to proceed executing cases until the last is reached. Proceeding through different cases is called <b>fall-through</b>. But, if no fall-through is desired, we can get out of <b>switch</b> statements with <b>break</b> or <b>return</b>.

Do not use <b>default</b> cases when handling <b>enumeration</b>! Leave the default out, so as to be able to catch enumerators not used in the statement, i.e., that were left out of the latter.

#### 9.4.2.1 - Declarations In Cases

It is possible to declare in cases, but every declaration <b>MUST</b> be initialized, and the cases where both declaration and initialization are <b>MUST</b> be executed. Not doing so will invoke a compiler error. An exception to this is the declaration of <b>int</b> entities. Since they need not be initialized, the declaration can happen without the latter. BUT, the USE cannot happen withou an initialization, because a compiler error will be happen. Anyways, AVOID handling uninitialized variables.

Also, it seems that, unless a block of code restricts a case, anything declared in it are in the scope of the switch statements, instead of a local scope of the case itself.

In [1]:
#include<string>
#include<iostream>

// The switch statement in wrong( int test ) will not run. This is so because case 0 might happen
// to be bypassed. If the code goes straight to case 1, shit is going down
/*void wrong( int test ){
    
    switch( test ){
            
        case 0:
            
            int x;
            int y = 3;
            
            std::string s;
            
        case 1:
            
            ++x;
            ++y;
            
            s = "nasty";
    }
}*/

// This, however, will run. This is so because x,y and text are strictly case-dependent, i.e.,
// their declaration is confined to the block of code, or scope, in which they were declared
void right( int value ){
    
    switch( value ){
            
        case 0:{
            
            int x = 0;
            
            ++x;
            
            std::cout << x;
            }
            
        case 1:{
            
            int y = 3;
            
            ++y;
        }
            
        case 2:{
            
            std::string text{ };
            
            text = "aight";
        }
    }
}

right( 0 );

1

### 9.4.3 - Declarations In Conditions

We can declare <b>INSIDE</b> conditions. More precisely, we can do the following:

In [ ]:
if( double example = prim( true ) ){
    
    break;
}

The declaration happens at condition opening. Its value is provided and tested within the condition, and its scope extends to the end of the block corresponding to the if-statement, i.e., it cannot be accessed outside the latter.